# 논문 (git commit log 활용)
## 프로젝트 git 대상 파일 목록을 메뉴별로 mapping

In [ ]:
pip install jupyterlab-kite

In [1]:
import pandas as pd
import re
import os
import subprocess
from tabulate import tabulate

## 1-1. xml파일에서 .do 포함하는 파일 추출하기
### findstr /s  action.*\.do D:\sanhakin\src\main\*.xml > xml_do_list.txt

In [2]:
# xml_do 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
command = 'findstr /s action.*\\.do D:\\sanhakin\\src\\main\\*.xml > ./txt/xml_do_list.txt'
try:
    subprocess.run(command, shell=True, check=True)
    print("명령어가 성공적으로 실행되었습니다.")
except subprocess.CalledProcessError as e:
    print("오류 발생:", e)

data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open("./txt/xml_do_list.txt", "r", encoding="utf-8") as file:
    sch_txt = 'action="/sanhakin'

    for line in file:
        # 각 줄을 스페이스로 분리하여 데이터를 추출합니다.
        parts = line.strip().split()
        xml_url = parts[0].replace("D:\\sanhakin\\src\\main\\webapp", "").replace("\\", "/").rsplit(':', 1)[0].rstrip()
        # xml_url = parts[0].replace("D:\\sanhakin\\src\\main\\webapp", "").replace("\\", "/")[:-1]
 
        for part in  parts[1:]:
            idx = part.find(sch_txt)
            if idx > -1:
                 do_txt = part[idx + len(sch_txt):len(part)-1]
                 # print("do_txt-->" +  do_txt)
                 data.append({'xml_url': xml_url, 'Do_txt': do_txt})
                 break

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/xml_do.csv', encoding="utf-8", index=False)

명령어가 성공적으로 실행되었습니다.


## 1-2. xml파일에서 .xml 포함하는 파일 추출하기 (화면에서 다른 메뉴화면 call 하는 경우) //주석으로 된 줄 제외
### findstr /s  \/wqxml.*\.xml D:\sanhakin\src\main\*.xml | findstr /v /i /c:"//" > xml_Inxml.txt 


In [3]:
# xml_do 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
command = 'findstr /s  \/wqxml.*\\.xml D:\\sanhakin\\src\main\\*.xml | findstr /v /i /c:"//" > ./txt/xml_Inxml.txt'
try:
    subprocess.run(command, shell=True, check=True)
    print("명령어가 성공적으로 실행되었습니다.")
except subprocess.CalledProcessError as e:
    print("오류 발생:", e)
    
data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open('./txt/xml_Inxml.txt', 'r', encoding='utf-8') as file:

    for line in file:
        # 각 줄을 스페이스로 분리하여 데이터를 추출합니다.
        parts = line.strip().split()
        xml_url = parts[0].replace('D:\\sanhakin\\src\\main\\webapp', '').replace('\\', '/').rsplit(':', 1)[0].rstrip()
        
        #두번째 원소부터 /wqxml/과 .xml 을 포함하는지 체크해서 문자열 추출하기
        for part in parts[1:]:
            if '/wqxml/' in part:
                start_index = part.find('/wqxml/')
                end_index = part.find('.xml', start_index)
                
                if start_index != -1 and end_index != -1:
                    inXmlPath = part[start_index:end_index + 4]
                    # print("inXmlPath String:", inXmlPath)
                    # print("xml_path-->" +  xml_path)
                    data.append({'xml_url': xml_url, 'inXml': inXmlPath})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df = df.drop_duplicates()
# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/xml_Inxml.csv', encoding="utf-8", index=False)

명령어가 성공적으로 실행되었습니다.


## 2. _Controller.java 파일에서 .do 포함 텍스트 추출
### findstr /s  \@RequestMapping\(.*\) D:\sanhakin\src\main\*Controller.java > controller_do.txt

In [4]:
# do_controller 파일을 읽어서 정제 후 do 포함 텍스트와 controller명을 csv 파일로 저장
command = 'findstr /s  \@RequestMapping\(.*\) D:\\sanhakin\\src\\main\\*Controller.java > ./txt/controller_do.txt'
try:
    subprocess.run(command, shell=True, check=True)
    print("명령어가 성공적으로 실행되었습니다.")
except subprocess.CalledProcessError as e:
    print("오류 발생:", e)
    
# 빈 리스트를 생성하여 데이터 저장 준비
data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open("./txt/controller_do.txt", "r", encoding="utf-8") as file:
    for line in file:
        # 각 줄을 '@RequestMapping'로 분리하여 데이터를 추출합니다.
        parts = line.split('@RequestMapping')
        
        if not any(part.startswith("D:\\sanhakin\\") for part in parts):
            continue
        
        controller_url = parts[0].replace("D:\\sanhakin\\", "").replace("\\", "/").rsplit(':', 1)[0].rstrip()

        #경로와 파일이름 분리
        controller_path, controller_file = os.path.split(controller_url)
        controller_path = os.path.dirname(controller_path)
        
        # 'ctrl'로 분리해서 controller.java 경로 추출
        # controller_path_file = controller_url.split('/ctrl/')
        
        idx = parts[1].find(",")
        if idx >  -1:
            for text in parts[1].split(","):
                do_txt_match = re.search(r'"([^"]*)"', text)
                do_txt = do_txt_match.group(1) if do_txt_match else None
                data.append({'controller_url': controller_url, 'controller_path':controller_path, 'controller_file': controller_file, 'Do_txt': do_txt})
        else:
            do_txt_match = re.search(r'"([^"]*)"', parts[1])
            do_txt = do_txt_match.group(1) if do_txt_match else None
            data.append({'controller_url': controller_url, 'controller_path':controller_path, 'controller_file': controller_file, 'Do_txt': do_txt})
                
# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/controller_do.csv', encoding="utf-8", index=False)

명령어가 성공적으로 실행되었습니다.


## 3. _DAO.java 파일에서 sql_id 텍스트(" "로 묶인 문자열) 추출 
### findstr /s /i /r /c:"\".*\"" "D:\sanhakin\src\main\*DAO.java" | findstr /v /i /c:"@SuppressWarnings" | findstr /v /i /c:"@Repository"  > DAO_sql.txt 

In [6]:
# DAO_sql.txt(최초 search 목록에서 eyeCheck로 일부 정제) 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
# D:\sanhakin\src\main\java\com\wti\admin\bizDvr\dao\BudgetDvrRqstSttnDAO.java:        return (List<Map>) list("budgetDvrRqstSttnDAO.selectBudgetDvrRqstSttnList", input);
#command = r'findstr /s /i /r /c:"\".*\"" "D:\sanhakin\src\main\*DAO.java" | findstr /v /i /c:"@SuppressWarnings" | findstr /v /i /c:"@Repository" > DAO_sql.txt'
command = (
    r'findstr /s /i /r /c:"\".*\"" "D:\sanhakin\src\main\*DAO.java" '
    r'| findstr /v /i /c:"@SuppressWarnings" '
    r'| findstr /v /i /c:"@Repository" > ./txt/DAO_sql.txt'
)
try:
    subprocess.run(command, shell=True, check=True)
    print("명령어가 성공적으로 실행되었습니다.")
except subprocess.CalledProcessError as e:
    print("오류 발생:", e)
    
data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open("./txt/DAO_sql.txt", "r", encoding="utf-8") as file:
    for line in file:
        # 각 줄을 스페이스로 분리하여 데이터를 추출합니다.
        parts = line.strip().split()

        if not any(part.startswith("D:\\sanhakin\\") for part in parts):
            continue
        
        DAO_url = parts[0].replace("D:\\sanhakin\\", "").replace("\\", "/").rsplit(':', 1)[0].rstrip()

        #경로와 파일이름 분리
        DAO_path, DAO_file = os.path.split(DAO_url)
        DAO_path = os.path.dirname(DAO_path)
        ## 'dao'로 분리해서 DAO.java 경로 추출
        # DAO_path_file = DAO_url.split('/dao/')
        
        # if len(DAO_path_file) == 2:
        #     DAO_path, DAO_file = DAO_path_file
        # else:
        #     # Handle the case where the split didn't result in two parts
        #     DAO_path, DAO_file = DAO_path_file[0], None

        sqlID_match = re.search(r'"([^"]*)"', line)
        sqlID = sqlID_match.group(1) if sqlID_match else None

        # Append data to the list
        data.append({'DAO_url': DAO_url, 'DAO_path': DAO_path, 'DAO_file': DAO_file, 'sqlID': sqlID})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/DAO_sqlID.csv', encoding="utf-8", index=False)

명령어가 성공적으로 실행되었습니다.


## 4. _SQL.xml 파일에서 sqlID 추출 ( "<" ,"id=" ,">" 포함하는 문자열)
### SQLxml_sqlID.txt  ( sqlxml - sqlID )  sqlxml_sql.txt (2차가공) --> DAO_sqlID.csv

### findstr /s /r /c:"\<.*\s*id\s*\=\s*\>" D:\sanhakin\src\main\*_SQL.xml  > SQLxml_sqlID.txt


In [ ]:
data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open("./txt/SQLxml_sqlID.txt", "r", encoding="utf-8") as file:

    for line in file:
        # 각 줄을 스페이스로 분리하여 데이터를 추출합니다.
        parts = line.strip().split()
        SQLxml_url = parts[0]
        SQLxml_url = SQLxml_url.replace("D:\\sanhakin\\", "" ).replace("\\", "/").rsplit(':', 1)[0].rstrip()

        for part in  parts[1:]:
            #count = part.count('"')
            # 다음 문자로 시작하는 문자열은 제외
            if (part.startswith("parameterClass") or part.startswith("resultClass") or part.startswith("refid") or part.startswith("java\.util\.")):
                continue
            else :
                fst_idx = part.find('"')
                if fst_idx > -1:
                    snd_idx = part.find('"', fst_idx + 1)
                    sqlID = part[fst_idx + 1:snd_idx]
                    # print("SQLxml_url-->" +  SQLxml_url + " slqID-->"+slqID)
                    data.append({'SQLxml_url': SQLxml_url, 'sqlID': sqlID})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/SQLxml_sqlID.csv', encoding="utf-8", index=False)

## 5. txt 파일 읽어서 csv 파일로 저장 (데이터 전처리)
### service, serviceImpl, DAO, Controller, _SQL.xml, 화면 xml
#### [ dir /s /b D:\sanhakin\src\main\*service.java > service.txt  ]
#### [ dir /s /b D:\sanhakin\src\main\*serviceImpl.java > serviceImpl.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*DAO.java > DAO.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*Controller.java > Controller.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*_SQl.xml > slqXml.txt ]
#### [ dir /s /b D:\sanhakin\src\main\weuare\*.xml > xm.txt ]

In [ ]:
import csv

txt_paths = ['./txt/service.txt', './txt/serviceImpl.txt', './txt/DAO.txt', './txt/Controller.txt', './txt/slqXml.txt', './txt/xml.txt']
csv_paths = ['./csv/service.csv', './csv/serviceImpl.csv', './csv/DAO.csv', './csv/Controller.csv', './csv/slqXml.csv', './csv/xml.csv']
file_tys = ['service', 'impl', 'dao', 'ctrl', 'sql', 'xml']

for i, (txt_path, csv_path) in enumerate(zip(txt_paths, csv_paths)):
    file_ty = file_tys[i]
    
    # Read the text file and write its contents to a CSV file
    with open(txt_path, 'r') as txt_file, open(csv_path, 'w', newline='') as csv_file:
        # Assuming your text file has lines with values separated by spaces
        reader = csv.reader(txt_file, delimiter=' ')
        writer = csv.writer(csv_file)
        
        # header = [prefix + '_path', 'java_file']
        header = ['file_url', 'file_path', 'file_name', 'file_ty']
        writer.writerow(header)
        
        # Write data from the text file to the CSV file
        for row in reader:
            # sevice.java 경로 추출하기
            java_url = row[0].replace("D:\\sanhakin\\", "").replace('\\','/')
            #경로와 파일이름 분리
            java_path, java_file  = os.path.split(java_url)
            java_path = os.path.dirname(java_path)

            # java_path_file = java_url.split(split_txt)
            # if len(java_path_file) == 2:
            #     java_path, java_file = java_path_file
            # else:
            #     # Handle the case where the split didn't result in two parts
            #     java_path, java_file = java_path_file[0], None

            writer.writerow([java_url, java_path, java_file, file_ty])
            
    print(f"Conversion from {txt_path} to {csv_path} completed.")

## 5-2 관리대상 파일 하나로 합쳐서 csv 파일로 저장 --> 데이터베이스 wtc_src_file table에 저장

In [ ]:
# java 파일 목록
service_df = pd.read_csv("./csv/service.csv", encoding="utf-8") #251
serviceImpl_df = pd.read_csv("./csv/serviceImpl.csv", encoding="utf-8") #369
DAO_df = pd.read_csv("./csv/DAO.csv", encoding="utf-8") #276
ctrl_df = pd.read_csv("./csv/Controller.csv", encoding="utf-8") #297
slqXml_df = pd.read_csv("./csv/slqXml.csv", encoding="utf-8") #323
xml_df = pd.read_csv("./csv/xml.csv", encoding="utf-8") #714

src_files_df = pd.concat([service_df, serviceImpl_df, DAO_df, ctrl_df, slqXml_df, xml_df], ignore_index=True)

src_files_df = src_files_df.drop_duplicates()
src_files_df.fillna('')

src_files_df.sort_values(by=['file_path','file_ty'], ascending=False)

# 'file_id' 컬럼 추가하고 규칙에 따라 file_id 값 부여
src_files_df['file_id'] = ['SRC_{:06}'.format(i+1) for i in range(len(src_files_df))]
src_files_df.to_csv('./csv/src_files.csv', encoding="utf-8", index=False) # wtc_src_file table에 저장

## 6. csv 파일 읽기

In [ ]:
# csv 파일 불러오기. 인코딩은 환경에 따라 변경 필요. 일반적으로 encoding= 'utf-8' 사용
menuID_xml = pd.read_csv("./csv/menu_xml.csv", encoding="euc-kr")
xml_Inxml = pd.read_csv("./csv/xml_Inxml.csv", encoding="utf-8")

controller_do = pd.read_csv("./csv/controller_do.csv", encoding="utf-8") #'controller_url', 'Do_txt'
xml_do = pd.read_csv("./csv/xml_do.csv", encoding="utf-8")               # xml_nm', 'Do_txt

DAO_sqlID = pd.read_csv("./csv/DAO_sqlID.csv", encoding="utf-8")         # DAO_url', 'slqID
SQLxml_sqlID = pd.read_csv("./csv/SQLxml_sqlID.csv", encoding="utf-8")   # SQLxml_url', 'slqID'

# java 파일 목록   ['service', 'impl', 'dao', 'ctrl', 'sql', 'xml']
src_files_df = pd.read_csv("./csv/src_files.csv", encoding="utf-8")

ctrl_df = src_files_df[src_files_df['file_ty']=='ctrl']
service_df = src_files_df[src_files_df['file_ty']=='service']
serviceImpl_df = src_files_df[src_files_df['file_ty']=='impl']
DAO_df = src_files_df[src_files_df['file_ty']=='dao']

# commit 정보
commitInfo_df = pd.read_csv("./csv/commit_info.csv", encoding="utf-8")
# commit 파일 정보
commitFile_df = pd.read_csv("./csv/commit_file_info.csv", encoding="utf-8")

#화면 내에서 콜하는 xml 목록에서 메인xml의 menu_id를 찾아서 맵핑
inXml_df = pd.merge(xml_Inxml, menuID_xml, on=['xml_url'], how='inner')[['menu_id','inXml']]
inXml_df.rename(columns={'inXml': 'xml_url'}, inplace = True)

# # 모든 xml - menu_id 합치기
merge_xml_menu_df =  pd.concat([menuID_xml, inXml_df], ignore_index=True)
merge_xml_menu_df = merge_xml_menu_df.drop_duplicates()
merge_xml_menu_df.info()
merge_xml_menu_df.head()

## 7. java 파일 경로 분리해서 컬럼 추가
### txt 파일 읽어서 csv 파일로 저장할 때 분리해서 저장하도록 수정해서 이 과정은 pass

## 8-1. merge 파일별 menu_id 맵핑하기
### menuID_xml   xml_do  controller_do   DAO_sqlID   SQLxml_sqlID

In [ ]:
# controller_do.info() #1447   controller_path   controller_file  Do_txt
# service_df.info() #250        service_path        java_file
# serviceImpl_df.info #368    serviceImpl_path    java_file
# DAO_df.info #275            dao_path            java_file
# SQLxml_sqlID.info()

In [ ]:
# 최종 맵핑을 위한 사전 dataFrame
# controller RequestMap 과 xml 화면에서 submition 의 .do 와 맵핑
controller_xml_df = pd.merge(controller_do, xml_do, on=['Do_txt'], how='inner')
controller_xml_df = controller_xml_df.drop_duplicates(subset=['controller_url','xml_url' ])

# DAO 에 있는 sqlId 와 SQL.xml 에 있는 sqlId 맵핑
dao_sql_df = pd.merge(DAO_sqlID, SQLxml_sqlID, on=['sqlID'], how='inner')
dao_sql_df = dao_sql_df.drop_duplicates(subset=['DAO_url','SQLxml_url' ])

In [ ]:
# menu_id 에 등록된 화면 xml 과 controller_xml_df의 xml 과 맵핑
# 0. 화면속 call xml파일
merge_xml_menu_df.info()

# 1. controller java
merge_xml_ctrl = pd.merge( controller_xml_df, merge_xml_menu_df, on='xml_url', how='left')
merge_xml_ctrl.dropna(subset = ['menu_id'], inplace=True)

# java 경로로 조인해서 menu_id 가져와서 dataFrame 만들기
# 1-2. controller java
ctrl_menu_df = pd.merge(ctrl_df, merge_xml_ctrl, left_on='file_path', right_on='controller_path', how='left')[['file_url', 'menu_id']]
ctrl_menu_df.dropna(subset = ['menu_id'], inplace=True)
ctrl_menu_df = ctrl_menu_df.drop_duplicates()

# 2. service java
service_menu_df = pd.merge(service_df, merge_xml_crtl, left_on='file_path', right_on='controller_path', how='left')[['file_url', 'menu_id']]
service_menu_df.dropna(subset = ['menu_id'], inplace=True)
service_menu_df = service_menu_df.drop_duplicates()

# # 3. serviceImpl java
serviceImpl_menu_df = pd.merge(serviceImpl_df, merge_xml_crtl, left_on='file_path', right_on='controller_path', how='left')[['file_url', 'menu_id']]
serviceImpl_menu_df.dropna(subset = ['menu_id'], inplace=True)
serviceImpl_menu_df = serviceImpl_menu_df.drop_duplicates()

# # 4. DAO java
DAO_menu_df = pd.merge(DAO_df, merge_xml_crtl, left_on='file_path', right_on='controller_path', how='left')[['file_url', 'menu_id']]
DAO_menu_df.dropna(subset = ['menu_id'], inplace=True)
DAO_menu_df = DAO_menu_df.drop_duplicates()

# # controller 경로 -  DAO.java - sqlID - SQL.xml
# # 5. _SQL.xml
sql_menu_df = pd.merge(dao_sql_df, merge_xml_crtl, left_on='DAO_path', right_on='controller_path', how='left')[['DAO_url','DAO_path', 'SQLxml_url', 'menu_id']]
sql_menu_df.dropna(subset = ['menu_id'], inplace=True)
sql_menu_df = sql_menu_df.drop_duplicates()

## 8-2 메뉴별 파일 합치고  src_files_df 과 merge 해서 file_id mapping하기

In [ ]:
# merge_xml_menu_df 424 menu_id  xml_url  
# ctrl_menu_df #1397         file_url  	menu_id
# service_menu_df #1436      file_url  	menu_id
# serviceImpl_menu_df #1544  file_url  	menu_id
# DAO_menu_df #1408          file_url  	menu_id
# sql_menu_df  #1556          DAO_url    DAO_path  SQLxml_url  menu_id

In [ ]:
# menuID_java_df = pd.concat([ctrl_menu_df, service_menu_df, serviceImpl_menu_df, DAO_menu_df], ignore_index=True)
menuID_java_df.descrypt
menuID_java_df.head()

# menuID_fileURL_df = pd.concat([merge_xml_menu_df, ctrl_menu_df, service_menu_df, serviceImpl_menu_df, DAO_menu_df, sql_menu_df], ignore_index=True)

## [ 최종 ] 메뉴 ID 에 해당하는 파일목록과 commit log 파일 목록 join 해서 찾기

In [ ]:
# csv 파일 불러오기. 인코딩은 환경에 따라 변경 필요. 일반적으로 encoding= 'utf-8' 사용
sanhakin_file_df = pd.read_csv("./csv/sanhakin_file.csv", encoding="utf-8") #java_url file_ty   menu_id   
commit_info_df = pd.read_csv("./csv/commit_info.csv", encoding="utf-8") 
commit_file_info_df = pd.read_csv("./csv/commit_file_info.csv", encoding="utf-8")

In [ ]:
sanhakin_file_df.head(10)

In [ ]:
# pd.set_option('display.max_colwidth', None)
# 찾기
# search_term = 'src/main/resources/egovframework/sqlmap/com/usr/aply/idtcag/H3gReqCompInfo_SQL.xml'
# result_df = sanhakin_file_df[sanhakin_file_df['java_url'].str.contains(search_term)]

# Select rows from commit_file_info where 'file_path' matches the condition
menu_selected_files = commit_file_info_df[commit_file_info_df['file_path'].isin(sanhakin_file_df[sanhakin_file_df['menu_id'] == 'MENU0000000160']['java_url'])]
selected_commit = commit_info_df[commit_info_df['commit_id'].isin(menu_selected_files['commit_id'])]
selected_commit.sort_values(by='commit_date', ascending=False)
selected_commit.fillna('')


In [ ]:
commit_file_info_df[commit_file_info_df['commit_id'] == 'c337f5c43' ]

In [ ]:
merge_df = pd.merge( commit_info_df, commit_file_info_df, on='commit_id', how = 'inner')[['commit_id','commit_date','commit_cls','commit_summ','file_path']]
menu_merge_df = pd.merge( merge_df, sanhakin_file_df, left_on='file_path', right_on = 'java_url',how='inner')[['commit_id','commit_date','menu_id','menu_title','commit_cls','commit_summ','file_path']]
menu_merge_df.head(10)

In [ ]:
# menu_merge_df.groupby(['menu_id','menu_title']).count()
menu_merge_df['commit_date'] = pd.to_datetime(menu_merge_df['commit_date'])
# # Extract year and month into new columns
menu_merge_df['commit_year'] = menu_merge_df['commit_date'].dt.year
menu_merge_df['commit_month'] = menu_merge_df['commit_date'].dt.month

# grouped_df = menu_merge_df.groupby(['commit_year', 'commit_month','menu_id','menu_title']).agg(commit_id_count=('commit_id', 'nunique')).reset_index()
grouped_df = menu_merge_df.groupby(['commit_year', 'commit_month','commit_cls']).agg(commit_id_count=('commit_id', 'nunique')).reset_index()

# menu_merge_df[['cyl','wt']].groupby(['commit_year', 'commit_month','commit_cls']).count()
# Display the result or use it as needed
print(grouped_df)

In [ ]:
## 공부 참고
import pandas as pd
import os

controller_url = "some/path/to/your/directory/your_file.txt"

# Split the path into the directory and the file
controller_path, controller_file = os.path.split(controller_url)

# Get the directory path excluding the last directory name
directory_path_excluding_last = os.path.dirname(controller_path)

# Display the result
print("Directory Path Excluding Last Directory:", directory_path_excluding_last)

# 중복 체크 및 삭제---------------------
# Assuming df is your DataFrame and you want to check duplicates based on columns 'col1' and 'col2'
df = pd.DataFrame({
    'col1': [1, 2, 3, 4, 1],
    'col2': ['A', 'B', 'C', 'D', 'A'],
    'col3': ['X', 'Y', 'Z', 'W', 'X']
})

# Identify duplicate rows based on 'col1' and 'col2'
duplicates = df.duplicated(subset=['col1', 'col2'], keep=False)

# Display the duplicate rows
print(df[duplicates])

# Remove duplicate rows based on 'col1' and 'col2'
df_no_duplicates = df.drop_duplicates(subset=['col1', 'col2'])

# Display the DataFrame without duplicates
print(df_no_duplicates)


## 중복체크
duplicate_values = DAO_sqlID[DAO_sqlID.duplicated(subset='sqlID', keep=False)]
print("DataFrame with Duplicate Values:")
print(duplicate_values)
## null 값 삭제
controller_do.dropna(subset=['controller_file'], inplace=True)
controller_do.describe()